In [2]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv,find_dotenv
from openai import OpenAI
import gradio as gr

In [4]:
_=load_dotenv(find_dotenv())
openai = OpenAI()

In [5]:
system_message = "You are a helpful assistant"

In [6]:
MODEL="gpt-4o-mini"
def message_gpt(prompt):
    messages=[
        {"role":"system","content":system_message},
        {"role":"user","content":prompt}
    ]
    c=openai.chat.completions.create(
        messages=messages,
        model=MODEL
    )
    return c.choices[0].message.content

In [7]:
message_gpt("What is today's date?")

"Today's date is October 9, 2023."

In [11]:
def shout(t:str):
    return t.upper()
gr.Interface(fn=shout,inputs="textbox",outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [14]:
ui=gr.Interface(fn=shout,inputs="textbox",outputs="textbox",allow_flagging="never")
ui.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://0f62e8549dbe4a34ec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
v=gr.Interface(
    inputs=[gr.Textbox(label="input",lines=6)],
    outputs=[gr.Textbox(label="output",lines=6)],
    fn=shout,
)
v.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [20]:
v=gr.Interface(
    inputs=[gr.Textbox(label="Message",lines=6)],
    outputs=[gr.Markdown(label="Output")],
    fn=message_gpt,
    allow_flagging="never"
)
v.launch()

/home/vboxuser/repos/llm/edDonner/venv/lib/python3.10/site-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [28]:
def stream_gpt(user_input:str):
    s=openai.chat.completions.create(
        messages=[
            {"role":"system","content":"You are a helpful assistant that responds in markdown"},
            {"role":"user","content":user_input}
        ],
        model=MODEL,
        stream=True
    )
    result=""
    for chunk in s:
        result+=chunk.choices[0].delta.content or ""
        # yield chunk.choices[0].delta.content # This will print one chunk in UI and will not get full text at once, try uncomment & see
        yield result

In [29]:
v=gr.Interface(
    inputs=[gr.Textbox(label="Message",lines=6)],
    outputs=[gr.Markdown(label="Output")],
    fn=stream_gpt,
    allow_flagging="never"
)
v.launch()

/home/vboxuser/repos/llm/edDonner/venv/lib/python3.10/site-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [42]:
def select_model(prompt,model):

    print(prompt,model)
    if model=="GPT":
        result=stream_gpt(prompt)
    else:
        raise ValueError("Unknown Model")
    for chunk in result:
        yield chunk

In [43]:
view = gr.Interface(
    fn=select_model,
    inputs=[
        gr.Textbox(label="Your message:"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")
    ],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never"
)
view.launch()


/home/vboxuser/repos/llm/edDonner/venv/lib/python3.10/site-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


how to reach india from Mars GPT


In [44]:
class Website():
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self,url):
        self.url=url
        response=requests.get(url)
        self.body=response.content
        if response.status_code==200 and self.body!="":
            soup = BeautifulSoup(self.body,'html.parser')
            self.title = soup.title.string if soup.title !="" else "No title found"
            for irrelavent in soup.body(["script", "style", "img", "input"]):
                irrelavent.decompose()
            self.text=soup.body.get_text(strip=True, separator='\n')
            links=[link.get('href') for link in soup.find_all('a')]
            self.links= [link for link in links if link]
        else:
            self.title =""
            self.text=""
    def get_contents(self):
         return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [51]:
system_prompt="You are an assistant, helping to find relevant link from given website content and \
with help of these links you will create company brochure for future investors, partners and recruit. Respond in markdown"

def stream_brochure(company_name, url,tone):
    prompt=f"You have to create a company borchure for {company_name}. Use {tone} tone. Here website's text:"
    prompt+=Website(url).get_contents()
    s=openai.chat.completions.create(
        messages=[
            {"role":"system","content":system_prompt},
            {"role":"user","content":prompt}
        ],
        model=MODEL,
        stream=True
    )
    result=""
    for chunk in s:
        result+=chunk.choices[0].delta.content or ""
        # yield chunk.choices[0].delta.content # This will print one chunk in UI and will not get full text at once, try uncomment & see
        yield result

In [52]:
gr.Interface(
    fn=stream_brochure,
    outputs=[gr.Markdown(label="Output")],
    inputs=[gr.Text(label="company_name"),gr.Textbox(label="Landing page URL:"),gr.Dropdown(["Formal","Funny"],label="Tone")],
    allow_flagging="never"
).launch()

/home/vboxuser/repos/llm/edDonner/venv/lib/python3.10/site-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.
